In [ ]:
from importlib.metadata import version
pkgs = [
	"huggingface_hub",  # to download pretrained weights
	"sentencepiece",    # to implement the tokenizer
	"torch",            # to implement the model
]
for p in pkgs:
	print(f"{p} version: {version(p)}")

huggingface_hub version: 0.33.2
sentencepiece version: 0.2.0
torch version: 2.4.1


### 1 GPT $\rightarrow$ LLama2

#### 1.1 LayerNorm $\rightarrow$ RMSNorm

- LayerNorm 用mean 和 variance 计算标准化

$$y_i = \frac{x_i - \mu}{\sqrt{\sigma^2 + \epsilon}} \gamma_i + \beta_i, \quad \text{where} \quad \mu = \frac{1}{n} \sum x_i, \quad \sigma^2 = \frac{1}{n} \sum (x_i - \mu)^2$$

- RMSNorm 用mean square 计算标准化的平方根

$$y_i = \frac{x_i}{\text{RMS}(x)} \gamma_i, \quad \text{where} \quad \text{RMS}(x) = \sqrt{\epsilon + \frac{1}{n} \sum x_i^2}$$



In [ ]:
import torch
import torch.nn as nn

# class LayerNorm(nn.Module):
#     def __init__(self, emb_dim):
#         super().__init__()
#         self.eps = 1e-5
#         self.scale = nn.Parameter(torch.ones(emb_dim))
#         self.shift = nn.Parameter(torch.zeros(emb_dim))

#     def forward(self, x):
#         mean = x.mean(dim=-1, keepdim=True)
#         var = x.var(dim=-1, keepdim=True, unbiased=False)
#         norm_x = (x - mean) / torch.sqrt(var + self.eps)
#         return self.scale * norm_x + self.shift

class RMSNorm(nn.Module):
	def __init__(self, emb_dim, eps=1e-5):
		super().__init__()
		self.eps = eps
		self.emb_dim = emb_dim
		self.weight = nn.Parameter(torch.ones(emb_dim)).float()

	def forward(self, x):
		means = x.pow(2).mean(dim=-1, keepdim=True)
		x_normed = x * torch.rsqrt(means + self.eps)
		return (x_normed * self.weight).to(dtype=x.dtype)

In [15]:
torch.manual_seed(123)

example_batch = torch.randn(2, 3, 4)

rms_norm = RMSNorm(emb_dim=example_batch.shape[-1])
rmsnorm_pytorch = torch.nn.RMSNorm(example_batch.shape[-1], eps=1e-5)

assert torch.allclose(rms_norm(example_batch), rmsnorm_pytorch(example_batch))

$\lvert \text{input} - \text{other} \rvert \leq \texttt{atol} + \texttt{rtol} \times \lvert \text{other} \rvert$

#### 1.2 GELU $\rightarrow$ SiLU
$$ \text{GELU}(x) = x \cdot \Phi(x), \quad \text{where} \quad \Phi(x) = \frac{1}{2} \left(1 + \text{erf}\left(\frac{x}{\sqrt{2}}\right)\right) $$

$$ \mathrm{erf}(x) = \frac{2}{\sqrt{\pi}} \int_0^x e^{-t^2} , dt $$

$$
\text{silu}(x) = x \cdot \sigma(x), \quad \text{where} \quad \sigma(x) \text{ is the logistic sigmoid.}
$$

In [ ]:
class SiLU(nn.Module):
	def __init__(self) -> None:
		super().__init__()
		super(SiLU, self).__init__()

	def forward(self, x):
		return x * torch.sigmoid(x)

In [17]:
silu = SiLU()
assert torch.allclose(silu(example_batch), torch.nn.functional.silu(example_batch))

#### 1.3 FeedForward

$$\text{SwiGLU}(x) = \text{SiLU}(\text{Linear}_1(x)) * (\text{Linear}_2(x))$$

In [ ]:
class FeedForward(nn.Module):
	def __init__(self, cfg) -> None:
		super().__init__()
		self.fc1 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], biad=False)
		self.fc2 = nn.Linear(cfg["emb_dim"], cfg["hidden_dim"], dtype=cfg["dtype"], biad=False)
		self.fc3 = nn.Linear(cfg["hidden_dim"], cfg["emb_dim"], dtype=cfg["dtype"], biad=False)
		self.silu = SiLU()

	def forward(self, x):
		x_fc1 = self.fc1(x)
		x_fc2 = self.fc2(x)
		x = self.silu(x_fc1) * x_fc2	# element-wise multiplication
		return self.fc3(x)

#### 1.4 pos emb $\rightarrow$ RoPE

In [ ]:
from torch import device


def precompute_rope_params(head_dim, theta_base=10_000, context_length=4096):
	assert head_dim %2 == 0, "Embedding dimension must be even"	# 偶数

	# Compute the inverse frequencies
	inv_freq = 1.0 / (theta_base ** (torch.arange(0, head_dim, 2)[:(head_dim // 2)].float() / head_dim))

	# 下标
	positions = torch.arange(context_length)

	# angle
	angles = positions[:, None] * inv_freq[None, :]	# Shape: (context_length, head_dim // 2)
	angles = torch.cat([angles, angles], dim=1)	# Shape: (context_length, head_dim)

	cos = torch.cos(angles)
	sin = torch.sin(angles)

	return cos, sin

def compute_rope(x, cos, sin):
	# x: (batch_size, num_heads, seq_len, head_dim)
	batch_size, num_heads, seq_len, head_dim = x.shape
	assert head_dim % 2 == 0

	x1 = x[..., : head_dim // 2]
	x2 = x[..., head_dim // 2 :]

	cos = cos[:seq_len, :].unsqueeze(0).unsqueeze(0)	# Shape: (1, 1, seq_len, head_dim)
	sin = sin[:seq_len, :].unsqueeze(0).unsqueeze(0)

	# rotary transformation
	rotated = torch.cat((-x2, x1), dim=-1)	# 旋转90度
	x_roated = (x * cos) + (rotated * sin)

	return x_roated.to(dtype=x.dtype)

- 应用

In [21]:
batch_size = 2
context_len = 5
num_heads = 4
head_dim = 16

cos, sin = precompute_rope_params(head_dim=head_dim, context_length=context_len)

torch.manual_seed(123)
queries = torch.randn(batch_size, num_heads, context_len, head_dim)
keys = torch.randn(batch_size, num_heads, context_len, head_dim)
queries_rot = compute_rope(queries, cos, sin)
keys_rot = compute_rope(keys, cos, sin)

#### 1.5 添加RoPE 到attention

In [ ]:
class MultiHeadAttention(nn.Module):
	def __init__(self, d_in, d_out, context_length, num_heads, dtype=None):
		super().__init__()
		assert d_out % num_heads == 0, "d_out must be divisible by n_heads"

		self.d_out = d_out
		self.num_heads = num_heads
		self.head_dim = d_out // num_heads

		# 去掉bias
		self.W_query = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
		self.W_key = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
		self.W_value = nn.Linear(d_in, d_out, bias=False, dtype=dtype)
		self.out_proj = nn.Linear(d_out, d_out, bias=False, dtype=dtype)  # Linear layer to combine head outputs
		# self.dropout = nn.Dropout(dropout)
		self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

		cos, sin = precompute_rope_params(head_dim=self.head_dim, context_length=context_len)
		self.register_buffer("cos", cos)
		self.register_buffer("sin", sin)

	def forward(self, x):
		b, num_tokens, d_in = x.shape

		queries = self.W_query(x)	# Shape: (b, num_tokens, d_out)
		keys = self.W_key(x)
		values = self.W_value(x)

		# (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
		queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
		keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
		values = values.view(b, num_tokens, self.num_heads, self.head_dim)

		# (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
		queries = queries.transpose(1, 2)
		keys = keys.transpose(1, 2)
		values = values.transpose(1, 2)

		# 位置编码
		# GPT 的位置编码是在最开始的输入部分
		queries = compute_rope(queries, self.cos, self.sin)
		keys = compute_rope(keys, self.cos, self.sin)

		attn_scores = queries @ keys.transpose(2, 3)
		attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

		# (b, num_tokens, num_heads, head_dim)
		context_vec = (attn_weights @ values).transpose(1, 2)
		
		context_vec = context_vec.reshape(b, num_tokens, self.d_out)
		context_vec = self.out_proj(context_vec)

		return context_vec

In [24]:
batch_size = 1
context_len = 100
max_context_len = 4096
embed_dim = 128
num_heads = 4

example_batch = torch.randn((batch_size, context_len, embed_dim))

mha = MultiHeadAttention(
	d_in=embed_dim,
	d_out=embed_dim,
	context_length=max_context_len,
	num_heads=num_heads
)

mha(example_batch)

del mha  # delete to free up memory

#### 1.6 TransformerBlock

In [25]:
class TransformerBlock(nn.Module):
	def __init__(self, cfg):
		self.att = MultiHeadAttention(
			d_in=cfg["emb_dim"],
			d_out=cfg["emb_dim"],
			context_length=cfg["context_length"],
			num_heads=cfg["n_heads"],
			dtype=cfg["dtype"]
		)
		self.ff = FeedForward(cfg)

		self.norm1 = RMSNorm(cfg["emb_dim"])
		self.norm2 = RMSNorm(cfg["emb_dim"])
	
	def forward(self, x):
		shortcut = x	# 原始输入
		x = self.norm1(x)
		x = self.att(x)	# Shape [batch_size, num_tokens, emb_size]
		x = x + shortcut	# 原始输入 + feedforward

		shortcut = x	# 原始输入 + feedforward
		x = self.norm2(x)
		x = self.ff(x)
		x = x + shortcut	# 原始输入 + feedforward + attention

		return x

#### 1.7 Llama2

In [ ]:
class Llama2Model(nn.Module):
	def __init__(self, cfg):
		super().__init__()
		self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"], dtype=cfg["dtype"])
		
		self.trf_blocks = nn.Sequential(
			*[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
		)

		self.final_norm = RMSNorm(cfg["emb_dim"])
		
		self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False, dtype=["dtype"])
	
	def forward(self, in_idx):
		tok_embeds = self.tok_emb(in_idx)
		x = tok_embeds
		x = self.trf_blocks(x)
		x = self.final_norm(x)
		logits = self.out_head(x)
		return logits

AttributeError: module 'torch.nn' has no attribute 'Model'

### initialize model